In [10]:
# importing libraries
from langchain.llms import GooglePalm
import os
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
db_password=os.getenv("db_password")

In [6]:
# creating object of GooglePalm() class as llm
llm=GooglePalm(
    google_api_key=GOOGLE_API_KEY,
    temperature=0.4
)

In [7]:
# importing librariess for MYSQL database
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [11]:
# for database dependencies
db_user = "root"
db_password = db_password
db_host = "localhost"
db_name = "TIMS"

In [12]:
# calling SQLDatabase
db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
    )

In [ ]:
# table info
print(db.table_info)

In [ ]:
# creating object of SQLDatabaseChain
sql_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
Q_01 = sql_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")

In [ ]:
# Query-02 : The result is wrong
Q_02 = sql_chain.run("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
# the actual query is
# SELECT sum(tims.product.price*tims.product.stock_quantity)
# FROM tims.product WHERE (tims.product.size='S')
Q_02 = sql_chain.run(
    "SELECT sum(tims.product.price*tims.product.stock_quantity) FROM tims.product WHERE (tims.product.size='S')"
    )


In [ ]:
# Query-03  shows error
Q_03 = sql_chain.run(
    "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store"
    )

In [ ]:
# Query-03: The Actual Query
Q_03 = sql_chain.run(
    """SELECT sum((tims.product.price*tims.product.stock_quantity)-((tims.product.price*tims.product.stock_quantity)*(tims.discounts.pct_discount/100)))
FROM tims.product INNER JOIN tims.discounts ON
tims.product.pid=tims.discounts.pid
WHERE (tims.product.brand='Levi')"""
    )